In [1]:
#Importing Essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime
from sklearn.model_selection import train_test_split, KFold
import gc

In [3]:
#Importing Datasets
tr = pd.read_csv('train.csv')
ts = pd.read_csv('test.csv')
view = pd.read_csv('view_log.csv')
item = pd.read_csv('item_data.csv')

In [4]:
#Converting Impression_time field to datetime format in train and test dataset
tr['impression_time'] = pd.to_datetime(tr.impression_time)
ts['impression_time'] = pd.to_datetime(ts.impression_time)
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237609 entries, 0 to 237608
Data columns (total 7 columns):
impression_id      237609 non-null object
impression_time    237609 non-null datetime64[ns]
user_id            237609 non-null int64
app_code           237609 non-null int64
os_version         237609 non-null object
is_4G              237609 non-null int64
is_click           237609 non-null int64
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 12.7+ MB


### <font color='green'>Objective:</font>
> For the unique impression id in train and test dataset, get only the past user level aggreagtion data. i.e Get data from view_log dataset where server time of the view_log must be less than or equal to ad impression time.

### <font color='green'>Approach:</font>
> 1. Merge view_log and item dataset based on item_id

> 2. Merge train and test dataset (only columns impression_id, impression_time, user_id) with already merged view_log and item dataset based on user_id

> 3. From the merged output dataset in step2, filter rows where server time of the the view log must be less than or equal to ad impression time.

> 4. From the filtered dataset in step4, prepare aggregation data for each impression id

In [5]:
#Converting Impression_time field to datetime format in train and test dataset
view['server_time'] = pd.to_datetime(view.server_time)
#Anlaysing view_log dataset
view.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118622 entries, 0 to 3118621
Data columns (total 5 columns):
server_time    datetime64[ns]
device_type    object
session_id     int64
user_id        int64
item_id        int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 119.0+ MB


In [6]:
#Deriving dayoftheweek, month, hour and day from servertime in view_log
view['server_weekday'] = view.server_time.dt.weekday_name
view['server_month'] = view.server_time.dt.month
view['server_hour'] = view.server_time.dt.round('H').dt.hour
view['server_day'] = view.server_time.dt.day

#Merging view_log and item_data
view_item = view.merge(item, on='item_id')
view_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3116840 entries, 0 to 3116839
Data columns (total 14 columns):
server_time       datetime64[ns]
device_type       object
session_id        int64
user_id           int64
item_id           int64
server_weekday    object
server_month      int64
server_hour       int64
server_day        int64
item_price        int64
category_1        int64
category_2        int64
category_3        int64
product_type      int64
dtypes: datetime64[ns](1), int64(11), object(2)
memory usage: 356.7+ MB


In [7]:
tr_ts = pd.concat([tr[['impression_id', 'impression_time', 'user_id']],
                   ts[['impression_id', 'impression_time', 'user_id']]], axis='rows', sort=False)

tr_ts1 = tr_ts.merge(view_item, on='user_id', how='inner')

tr_ts2 = tr_ts1[tr_ts1.server_time<=tr_ts1.impression_time].copy()
del [[tr_ts, tr_ts1]]
gc.collect()

27

#### Creating new aggregation dataset for every user id based on merged Dataset

In [8]:
#How many device types did each user used?
#How many unique session_id's for each user in total?
#How many items & unique items did each user visit?
#What's the minimum, maximum & average price of all the items user visited?
#How many unique categories & product types did each user visited?
agg_col = {'device_type': ['nunique'], 'session_id':['nunique'], 'item_id': ['nunique', 'count'],
           'item_price': ['min', 'max', 'mean'], 'category_1': ['nunique'], 'category_2':['nunique'],
           'category_3': ['nunique'], 'product_type': ['nunique']
          }

imp_agg = tr_ts2.groupby(['impression_id']).agg(agg_col)
imp_agg.columns=['imp_' + '_'.join(col).strip() for col in imp_agg.columns.values]

In [10]:
#How many visits & unique items in each weekday, month, hour, day, category_1 & category_2?
col_l = ['server_weekday', 'server_month', 'server_hour', 'server_day', 'category_1', 'category_2']

for c in col_l:
    for x in tr_ts2[c].unique():
        tmp_map = tr_ts2[(tr_ts2[c]==x)].groupby(['impression_id']).size()
        cname1 = 'imp_'+ str(c) + '_' + str(x) +'_item_count'
        imp_agg[cname1] = imp_agg.index.map(tmp_map)
        imp_agg.loc[imp_agg[cname1].isnull(), cname1] = 0
        
        tmp_map = tr_ts2[(tr_ts2[c]==x)].groupby(['impression_id'])['item_id'].nunique()
        cname2 = 'imp_'+ str(c) + '_' + str(x) +'_item_nunique'
        imp_agg[cname2] = imp_agg.index.map(tmp_map)
        imp_agg.loc[imp_agg[cname2].isnull(), cname2] = 0

In [11]:
#Dropping merged dataset tr_ts2
del [[tr_ts2]]
gc.collect()

20

In [12]:
#Exporting aggregation dataset into imp_agg.csv file
imp_agg.reset_index().to_csv('imp_agg.csv', index=False)